In [1]:
from langchain_core.prompts import ChatPromptTemplate

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}")

In [3]:
prompt_val = prompt.invoke({"topic": "dog"})
print(prompt_val)

messages=[HumanMessage(content='Tell me an interesting fact about dog', additional_kwargs={}, response_metadata={})]


In [4]:
print(prompt_val.to_messages())

[HumanMessage(content='Tell me an interesting fact about dog', additional_kwargs={}, response_metadata={})]


In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")
result = model.invoke(prompt_val)
result

AIMessage(content='One interesting fact about dogs is that they have an extraordinary sense of smell, which is estimated to be between 10,000 to 100,000 times more sensitive than that of humans. This remarkable olfactory capability allows them to detect specific scents at incredibly low concentrations, making them indispensable in various roles, such as search and rescue, detecting drugs, and even identifying medical conditions like certain cancers or diabetes through scent. Their sense of smell plays a vital role in their communication and interaction with the world around them.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 14, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(result)

'One interesting fact about dogs is that they have an extraordinary sense of smell, which is estimated to be between 10,000 to 100,000 times more sensitive than that of humans. This remarkable olfactory capability allows them to detect specific scents at incredibly low concentrations, making them indispensable in various roles, such as search and rescue, detecting drugs, and even identifying medical conditions like certain cancers or diabetes through scent. Their sense of smell plays a vital role in their communication and interaction with the world around them.'

### Now let´s do this LCEL

In [8]:
prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}")
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

basicchain = model | output_parser

In [9]:
basicchain.invoke("hello!")

'Hello! How can I assist you today?'

In [10]:
chain = prompt | model | output_parser

chain.invoke({"topic": "dog"})

'One interesting fact about dogs is that they have an extraordinary sense of smell, which is estimated to be between 10,000 to 100,000 times more acute than that of humans. This incredible olfactory ability allows them to detect various scents, including those related to emotions, health conditions, and even certain diseases like cancer and diabetes. This unique skill is why dogs are often used in search-and-rescue operations, in detecting drugs, and in various therapeutic roles.'

### Most important methods of the Runnable Interface (Stream, Invoke, Batch)

In [12]:
for s in chain.stream({"topic": "bears"}):
    print(s, end="", flush=True)

An interesting fact about bears is that they have an incredible sense of smell—it's estimated to be seven times stronger than that of a bloodhound! This extraordinary olfactory capability allows them to detect food sources from great distances, sometimes even miles away. This keen sense of smell plays a crucial role in their hunting and foraging behavior, as they can locate carrion, berries, and other food sources with remarkable accuracy.

In [13]:
chain.invoke({"topic": "bears"})

"One interesting fact about bears is that they have an incredible sense of smell—up to seven times better than that of a bloodhound! This acute olfactory ability allows them to detect food from miles away and is essential for their survival, especially during foraging and finding mates. In fact, a bear's sense of smell can help them sniff out food sources that are buried deep under snow or even detect other bears from long distances."

In [14]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])

['Bears have an impressive sense of smell; it is estimated to be 7 times better than that of a bloodhound! This exceptional olfactory ability aids them in finding food over long distances, detecting predators, and even locating mates. Some research suggests that a bear can smell food from up to 20 miles away, making them incredibly adept at locating potential meals in their vast habitats.',
 'One interesting fact about cats is that they have a unique grooming behavior that serves multiple purposes. Cats spend a significant portion of their day—up to 50%—grooming themselves. This not only helps keep their fur clean and free of parasites but also has a calming effect, reducing stress and anxiety. Moreover, grooming helps distribute natural oils in their fur, keeping it healthy and waterproof. Additionally, when cats groom each other, a behavior known as allogrooming, it strengthens social bonds among them.']

In [15]:
async for s in chain.astream({"topic": "bears"}):
    print(s, end="", flush=True)

One interesting fact about bears is that they have an incredible sense of smell—up to seven times better than that of a bloodhound! This heightened olfactory ability allows them to detect food, locate mates, and sense danger from miles away. Some bears can smell food from over a mile away, which is why they are often successful foragers in the wild. Their sense of smell plays a critical role in their survival and behavior.

In [16]:
await chain.ainvoke({"topic": "bears"})

'One interesting fact about bears is that they are excellent swimmers! Many species of bears, including grizzly bears and polar bears, can swim for long distances in search of food or to cross rivers and lakes. Polar bears are particularly adapted to swimming, with large, paddle-like forelimbs and a thick layer of fat that helps keep them buoyant and insulated in icy waters. Some polar bears have been known to swim continuously for several days, covering distances of over 60 miles (97 kilometers) without resting!'

In [17]:
await chain.abatch([{"topic": "bears"}, {"topic": "cats"}])

['One interesting fact about bears is that they have an incredible sense of smell, which is significantly stronger than that of dogs. In fact, a bear’s sense of smell can be up to 7 times stronger than that of a bloodhound. This allows bears to detect food sources from miles away and can help them locate mates or avoid danger. Their keen sense of smell plays a crucial role in their survival and foraging behavior.',
 "One interesting fact about cats is that they have a unique collarbone (clavicle) structure that allows them to fit through tight spaces. Unlike dogs, cats' collarbones are not connected to other bones but are instead attached to the muscles of the shoulder. This anatomical feature gives them incredible flexibility and the ability to squeeze through openings as small as their head, which is one reason they are such agile hunters. This trait is also why they can often be seen maneuvering through narrow gaps and spaces around the house!"]

### Retrieval Augmented Generation with LCEL

In [20]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

embedding_function = OpenAIEmbeddings()

docs = [
    Document(
        page_content="the dog loves to eat pizza", metadata={"source": "animal.txt"}
    ),
    Document(
        page_content="the cat loves to eat lasagna", metadata={"source": "animal.txt"}
    ),
]


db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever()

In [21]:
retriever.invoke("What does the dog want to eat?")  # old method

[Document(metadata={'source': 'animal.txt'}, page_content='the dog loves to eat pizza'),
 Document(metadata={'source': 'animal.txt'}, page_content='the cat loves to eat lasagna')]

In [22]:
retriever.invoke("What does the dog want to eat?")

[Document(metadata={'source': 'animal.txt'}, page_content='the dog loves to eat pizza'),
 Document(metadata={'source': 'animal.txt'}, page_content='the cat loves to eat lasagna')]

In [23]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4o-mini")

In [24]:
from operator import itemgetter

retrieval_chain = (
    {
        "context": (lambda x: x["question"]) | retriever,
        # "question": lambda x: x["question"],
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [25]:
retrieval_chain.invoke({"question": "What does the dog like to eat?"})

'The dog likes to eat pizza.'

In [26]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4o-mini")

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [27]:
retrieval_chain.invoke("What does the dog like to eat?")

'The dog likes to eat pizza.'